### Trigger by a period-based rate curve

In [1]:
from absbox import API,Generic,EnginePath

In [2]:
test01 = Generic(
    "TriggerByPeriod"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":1800,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":1100
          ,"currentRate":0.08
          ,"remainTerm":30
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30},"feeStart":"2021-06-15"}),)
    ,{"amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["payInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,{
        "BeforeDistribution":{"periodTest":{
            "condition": ["periodRateCurve", ("cumPoolDefaultedRate",)
                                       , ">=", ("dealStat","int","BondPaidPeriod")
                                       ,[[0,0.014]
                                        ,[12,0.035]
                                        ,[24,0.06]]
                         ]
            ,"effects":("newStatus","Ended")
            ,"status":False
            ,"curable":False}
        }
    }
    ,("PreClosing","Amortizing")
    ,None
    ,None
    ,None
    ,{"BondPaidPeriod":0}
)

In [3]:
localAPI = API(EnginePath.DEV,check=False)

Connecting engine server -> https://absbox.org/api/dev

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅Connected, local lib:0.46.5, server:0.50.1

In [4]:
r = localAPI.run(test01
                 ,poolAssump = ("Pool",("Mortgage",{"CDR":0.02},None,None,None)
                                       ,None
                                       ,None)
                 ,read=True)

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


EngineError: Error in $.triggers.BeginDistributionWF.periodTest.trgEffects: key "contents" not found

In [22]:
r['triggers']['BeginDistributionWF']['periodTest']

,status,memo
date,,
2021-07-26,False,<Tag:Right 6.290909090909091e-3 GE Just 1.4e-2>
2021-08-20,False,<Tag:Right 7.736363636363637e-3 GE Just 1.4e-2>
2021-09-20,False,<Tag:Right 9.172727272727273e-3 GE Just 1.4e-2>
2021-10-20,False,<Tag:Right 1.0554545454545455e-2 GE Just 1.4e-2>
2021-11-20,False,<Tag:Right 1.1836363636363636e-2 GE Just 1.4e-2>
2021-12-20,False,<Tag:Right 1.3109090909090909e-2 GE Just 1.4e-2>
2022-01-20,True,<Tag:Right 1.4281818181818182e-2 GE Just 1.4e-2>
